In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
%matplotlib inline

In [2]:
data = pd.read_excel('HealthViz County Dataset 6.19.17.xlsx',skiprows=0, header=1, index_col=0)
data.index.name=None
#wgt = np.array(data['Population (residents), 2011-2015'])
data.drop(data.columns[[0,1]], axis=1, inplace=True)

In [5]:
#import pandas as pd
#df = pd.read_csv('HealthViz County Dataset 6 21 17.csv', skiprows=0, header=1, index_col=0,encoding = "ISO-8859-1")
#df.index.name=None

In [3]:
data.isnull().sum()

Median household income, 2011-2015                                                         3
Per capita income, 2011-2015                                                               2
Gini index of income inequality, 2015                                                   2390
Food stamps (SNAP) (% of households), 2011-2015                                            2
Poverty rate (% of residents), 2011-2015                                                   2
Median property tax ($), 2011-2015                                                        36
Rent-burdened (% of renter-occupied housing units), 2011-2015                              2
Median gross rent, 2011-2015                                                               3
Median home value, 2011-2015                                                               4
Occupied (% of housing units), 2011-2015                                                   2
Medicare flu vaccination rate (% of beneficiaries), 2015              

In [24]:
# Using linear regression to fill out the missing values in certain columns based on the rest of columns which have
# the full values. The fullfilled colunms will be considered as columns with full values to make prediction in the 
# later run.

# Step 0: Have the original dataset
# Step 1: Get the dataframe with all the columns with full values (Xs) and the colume with least missing value (Y)
# Step 2: Split the dataframe into trainning (Xs and Y with values) and predict datasets (Xs and Y with missing values)
# Step 3: Run the regression with the trainning set and get the predictions for the missing Y values in the predict set
# Step 4: The new dataframe has been created with the fullfillment of the Y
# Step 5: Merge the new dataframe with the other potential Ys to replace the orginal dataset to iterate the above steps
# until all the Ys have been fullfilled

def preprocess_df(data):
    '''
    Given a (controlled variables) dataframe, output a dataframe with the full-value columns (Xs) and the column with\
    least missing values (Y)
    
        data: dataframe to be dealt with
        Return: dataframe with only used Xs, full dataframe with used Xs and the Y, dataframe with the other Ys
    '''
    df_xs = data.dropna(axis=1, how='any')
    l_ys = data.columns[data.isnull().any()].tolist()
    len_l_ys = len(l_ys)
    i_min = 0
    df_y = pd.DataFrame(data[l_ys[i_min]])
    n_min = df_y.isnull().values.ravel().sum()
    for i in range(1, len_l_ys):
        df_y = pd.DataFrame(data[l_ys[i]])
        n = df_y.isnull().values.ravel().sum()
        if n < n_min:
            n_min = n
            i_min = i
    label_y = l_ys[i_min]
    df_y = pd.DataFrame(data[l_ys[i_min]])
    df = pd.merge(df_y, df_xs, right_index=True, left_index=True)
    df_ys = data[data.columns[data.isnull().any()]].drop(label_y, 1)
    return df_xs, df_ys, df


def train_predict_split(df):
    '''
    Given the full dataset, split it into train and predict datasets based on the missing value of the Y
    The first column of the dataset is the Y
    '''
    labels = list(df.columns)
    mask = df[labels[0]].isnull()
    df_predict = df[mask]
    df_train = df[~mask]
    y_train = df_train.ix[:, 0:1]
    x_train = df_train.ix[:, 1:]
    x_predict = df_predict.ix[:, 1:]
    y_predict = df_predict.ix[:, 0:1]
    return x_train, y_train, x_predict, df_predict, df_train
    
def new_predict_df(x_train, y_train, x_predict, df_predict, df_train, sample_weight = None):
    # sample_weight=x_train['Medicare flu vaccination rate (% of beneficiaries), 2015']
    '''
    Filling the missing values and return a new Xs and the Y dataframe
    '''
    lm = linear_model.LinearRegression()
    if sample_weight is not None:
        model = lm.fit(x_train,y_train, sample_weight=1/sample_weight)
    else:
        model = lm.fit(x_train,y_train)
    y_pred = lm.predict(x_predict)
    l = []
    d = {}
    for i in y_pred.tolist():
        l = l + i
    se = pd.Series(l)
    df_predict['prediction'] = se.values
    df_predict[y_train.columns.tolist()[0]].fillna(value=df_predict['prediction'], inplace=True)
    df_predict = df_predict.drop('prediction', 1)
    df = df_train.append(df_predict)
    return df

def impute_column(df, sample_weight=None):
    '''
    Given a data set, get the new precit data set with the predict values
    '''
    x_train, y_train, x_predict, df_predict, df_train = train_predict_split(df)
    df_new = new_predict_df(x_train, y_train, x_predict, df_predict, df_train, sample_weight=None)
    return df_new

def imputation(X,y, sample_weight=None):
    '''
    Given the dataset, return the whole dataset with the fullfillment of the missing values
    '''
    data = pd.merge(X, y, right_index=True, left_index=True)
    X_list = X.columns.tolist()
    y_list = y.columns.tolist()
    num_null = len(data.columns[data.isnull().any()].tolist())
    for i in range(0, num_null):
        df_xs, df_ys, df = preprocess_df(data)
        df_new = impute_column(df)
        data = pd.merge(df_ys, df_new, right_index=True, left_index=True)
    return data[X_list], data[y_list], sample_weight

In [25]:
data.columns.tolist()
X = data[['Median household income, 2011-2015',
 'Per capita income, 2011-2015',
 'Gini index of income inequality, 2015',
 'Food stamps (SNAP) (% of households), 2011-2015',
 'Poverty rate (% of residents), 2011-2015',
 'Median property tax ($), 2011-2015',
 'Rent-burdened (% of renter-occupied housing units), 2011-2015',
 'Median gross rent, 2011-2015',
 'Median home value, 2011-2015',
 'Occupied (% of housing units), 2011-2015',
 'Medicare flu vaccination rate (% of beneficiaries), 2015',
 'Race-Ethnicity Diversity Index, 2011-2015',
 'Median age, 2011-2015',
 'Married, Females (% of women ages 15-50), 2010-2014',
 'Birth rate, Females (births per 1,000 women ages 15-50), 2011-2015',
 'Teen birth rate, Females, Juveniles (5-17 years) (Births per 1,000 women), 2011-2015',
 'High school graduation rate (% of residents), 2011-2015',
 'Any higher education rate (% of residents), 2011-2015',
 'College graduation rate (% of residents), 2011-2015',
 'Unemployment rate (%), 2011-2015',
 'Dual eligible coverage (% of residents), 2010-2014',
 'Medicaid coverage (% of residents), 2010-2014',
 'Medicare coverage (% of residents), 2010-2014',
 'Private health insurance (% of residents), 2010-2014',
 'Uninsured rate (% of residents), 2011-2015',
 'Medicare Advantage participation rate (% of beneficiaries), 2015',
 'Medicare FFS per capita costs ($ per beneficiary), 2015',
 'Medicare emergency department visit rate (visits per 1,000 beneficiaries), 2015',
 'Readmission rate (Medicare) (% of hospital admissions), 2015',
 'Medicare inpatient utilization (% of beneficiaries), 2015',
 'Current cigarette smoking (% of adults), 2006-2012',
 'Diagnosed diabetes (% of adults), 2006-2012',
 'High blood pressure (% of adults), 2006-2012',
 'Obesity (% of adults), 2006-2012',
 'Mortality rate, all causes (deaths per 100,000), 2008-2014',
 'Diabetes mortality (deaths per 100,000), 2008-2014',
 'Influenza and pneumonia mortality (deaths per 100,000), 2008-2014',
 'Septicemia (sepsis) mortality (deaths per 100,000), 2008-2014']]
y = data[['Years of potential life lost (YPLL) (life-years lost per 100,000), 2012-2014']]
sample_weight=data['Medicare flu vaccination rate (% of beneficiaries), 2015']

In [28]:
sample_weight

1001     42.934783
1003     48.474011
1005     34.038662
1007     33.843058
1009     43.098782
1011     29.884275
1013     36.159263
1015     40.522842
1017     43.857934
1019     42.559916
1021     47.220723
1023     38.084489
1025     30.535629
1027     44.935471
1029     35.846030
1031     43.346442
1033     48.068366
1035     43.901501
1037     37.198516
1039     47.862343
1041     41.616766
1043     42.664109
1045     42.765438
1047     37.997125
1049     46.326146
1051     43.940635
1053     38.839741
1055     38.167038
1057     45.797379
1059     39.856718
           ...    
72095    24.060150
72097    19.013581
72099    18.885449
72101    17.391304
72103    15.555556
72105    17.406143
72107    15.865385
72109    12.500000
72111    15.000000
72113    21.108987
72115    14.642857
72117    20.975610
72119    19.512195
72121    13.019391
72123    21.367521
72125    18.043845
72127    26.251923
72129    19.471154
72131    20.464135
72133    22.285714
72135    15.669014
72137    16.

In [ ]:
X, y, sw = imputation(X,y, sample_weight)

/Users/admin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
/Users/admin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:51: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
/Users/admin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/admin/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:

In [22]:
X.isnull().sum()

Median household income, 2011-2015                                                      0
Per capita income, 2011-2015                                                            0
Gini index of income inequality, 2015                                                   0
Food stamps (SNAP) (% of households), 2011-2015                                         0
Poverty rate (% of residents), 2011-2015                                                0
Median property tax ($), 2011-2015                                                      0
Rent-burdened (% of renter-occupied housing units), 2011-2015                           0
Median gross rent, 2011-2015                                                            0
Median home value, 2011-2015                                                            0
Occupied (% of housing units), 2011-2015                                                0
Medicare flu vaccination rate (% of beneficiaries), 2015                                0
Race-Ethni

,"Median household income, 2011-2015","Per capita income, 2011-2015","Gini index of income inequality, 2015","Food stamps (SNAP) (% of households), 2011-2015","Poverty rate (% of residents), 2011-2015","Median property tax ($), 2011-2015","Rent-burdened (% of renter-occupied housing units), 2011-2015","Median gross rent, 2011-2015","Median home value, 2011-2015","Occupied (% of housing units), 2011-2015",...,"Readmission rate (Medicare) (% of hospital admissions), 2015","Medicare inpatient utilization (% of beneficiaries), 2015","Current cigarette smoking (% of adults), 2006-2012","Diagnosed diabetes (% of adults), 2006-2012","High blood pressure (% of adults), 2006-2012","Obesity (% of adults), 2006-2012","Mortality rate, all causes (deaths per 100,000), 2008-2014","Diabetes mortality (deaths per 100,000), 2008-2014","Influenza and pneumonia mortality (deaths per 100,000), 2008-2014","Septicemia (sepsis) mortality (deaths per 100,000), 2008-2014"
1001,51219.97561,24974.00000,0.443439,13.777211,12.879382,455.000000,49.407783,881.94923,141131.853,90.319724,...,16.760000,18.200000,21.000000,10.800000,27.300000,29.200000,917.200000,21.500000,17.700000,19.100000
1003,50194.19774,27317.00000,0.472200,9.003563,13.411661,573.000000,44.544464,877.95399,169098.533,69.632219,...,15.810000,16.230000,22.700000,7.500000,30.900000,25.000000,792.700000,17.500000,10.500000,10.000000
1005,32924.77284,16824.00000,0.476870,25.970505,26.727439,301.000000,40.946992,578.31099,92090.282,78.086367,...,18.440000,17.600000,25.200000,11.300000,46.400000,46.100000,914.300000,19.400000,12.000000,12.700000
1007,38631.97318,18431.00000,0.451918,15.781984,16.795878,266.000000,35.162950,650.22531,102577.787,78.330175,...,15.490000,17.810000,25.600000,9.200000,39.100000,30.500000,1029.300000,24.500000,17.000000,20.100000
1009,45758.48253,20532.00000,0.438260,13.489623,16.720933,419.000000,31.117687,600.28481,119657.438,87.242246,...,15.310000,15.980000,22.400000,8.600000,36.200000,34.400000,907.000000,13.800000,32.400000,20.800000
1011,31899.99378,17580.00000,0.446086,23.133315,24.618736,270.000000,38.361266,569.32170,68518.366,82.486002,...,18.150000,18.840000,40.100000,14.200000,25.800000,42.000000,948.100000,56.000000,0.000000,25.000000
1013,32190.64749,18390.00000,0.494741,23.460775,25.364736,311.000000,43.573798,574.31575,78806.109,81.217865,...,16.050000,19.870000,32.800000,10.100000,31.500000,35.400000,991.200000,44.600000,15.300000,17.800000
1015,41653.37343,21374.00000,0.421900,20.263985,20.532209,442.000000,45.252236,636.24197,105773.979,84.723056,...,16.940000,20.320000,25.600000,13.200000,36.200000,33.400000,1040.100000,13.500000,34.400000,30.200000
1017,34136.32937,21071.00000,0.480880,16.754914,21.589931,344.000000,45.424032,657.21698,80703.848,81.406471,...,15.250000,17.840000,25.300000,17.300000,44.700000,44.500000,1018.800000,47.800000,18.100000,18.100000
1019,36252.80776,21811.00000,0.487478,18.540521,19.229717,432.000000,41.517680,620.26101,104974.931,69.437261,...,20.120000,19.280000,27.800000,14.100000,43.600000,34.700000,1002.500000,15.700000,29.700000,13.900000
